In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../functions/')
from statistics_functions import *
import numpy as np

In [2]:
res = '4km'

path = '/work/bk1377/a270230/'
path_stat = path + 'statistics/'

In [3]:
# load the lkf data
data = np.load(path_stat + f'lkfs_paths_{res}.npz', allow_pickle=True)
years, lkfs, paths, paths_all = [data[key] for key in data.files]

LKFs = np.load(path_stat + f'LKFs_{res}.npy', allow_pickle=True)[0]

In [4]:
# calculate the mean of deformation, divergence, shear, and vorticity 
# of each lkf and add it to the dictionary
for year in years:
    
    y = np.where(years==year)[0][0]
    for d in range(365):

        eps_d, div_d, shr_d, vor_d = [], [], [], []
        for lkf in lkfs[y][d]:

            div_d += np.mean(lkf[:,4]),
            shr_d += np.mean(lkf[:,5]),
            vor_d += np.mean(lkf[:,6]),

        eps_d = np.sqrt(np.array(div_d)**2+np.array(shr_d)**2)

        LKFs[f'{year}'][f'{d+1}']['deformation'] = eps_d
        LKFs[f'{year}'][f'{d+1}']['divergence']  = div_d
        LKFs[f'{year}'][f'{d+1}']['shear']       = shr_d
        LKFs[f'{year}'][f'{d+1}']['vorticity']   = vor_d

In [5]:
# determine whether an lkf is a lead (1), a ridge (2) or not quantifiable (=longer lived) (3).
# "divergence<0 => lead" only works for the first day of the lkf. on the second day, a lead
# can have divergence>0 but is still a region of open water
for year in years:
    LKFs[f'{year}']['1']['lead or ridge'] = np.where(LKFs[f'{year}']['1']['divergence']<0, 1, 2)
    
    y = np.where(years==year)[0][0]
    for d in range(1,365):
        
        LKFs[f'{year}'][f'{d+1}']['lead or ridge'] = np.where(LKFs[f'{year}'][f'{d+1}']['divergence']<0, 1, 2)
        for lkf in range(len(lkfs[y][d])):
            
            for lkf_path in paths_all[0][0]:
                if np.size(lkf_path)>1:
                    if lkf_path[1] == lkf:
                        LKFs[f'{year}'][f'{d+1}'].loc[lkf, 'lead or ridge'] = 3
                        break

In [6]:
np.save(path_stat + f'LKFs_{res}', [LKFs])

In [7]:
LKFs['2013']['50']

,length,lifetime,growth rate,curvature,deformation,divergence,shear,vorticity,lead or ridge
0,241.516389,10,-50.034461,0.062930,0.466506,0.256067,0.389945,-0.130005,3
1,318.612405,10,-162.885809,0.149973,0.383111,0.062802,0.377929,0.162846,3
2,103.904072,5,2.897439,0.053494,0.057973,-0.033412,0.047376,0.013666,1
3,922.656429,2,-442.566406,0.060151,0.357876,-0.227370,0.276366,-0.030318,3
4,1912.828484,1,NaN,0.108156,0.207169,0.098249,0.182391,0.040061,3
...,...,...,...,...,...,...,...,...,...
76,146.306640,1,NaN,0.086637,0.085851,0.056291,0.064821,0.012501,2
77,91.403115,1,NaN,0.097042,0.036836,0.004920,0.036506,-0.015418,2
78,48.768880,4,316.926823,0.071365,1.737195,-0.902559,1.484330,-0.402298,1
79,95.304988,5,62.806455,0.052885,0.504432,0.355663,0.357708,0.019632,2
